In [11]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr
import numpy as np
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm


def vectorize_dic(dic,ix=None,p=None,n=0,g=0):
    """
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if ix==None:
        ix = dict()

    nz = n * g

    col_ix = np.empty(nz,dtype = int)

    i = 0
    for k,lis in dic.items():
        for t in range(len(lis)):
            ix[str(lis[t]) + str(k)] = ix.get(str(lis[t]) + str(k),0) + 1
            col_ix[i+t*g] = ix[str(lis[t]) + str(k)]
        i += 1

    row_ix = np.repeat(np.arange(0,n),g)
    data = np.ones(nz)
    if p == None:
        p = len(ix)
    print(len(col_ix),col_ix)
    print(len(row_ix),row_ix)
    print(len(ix),ix)
    ixx = np.where(col_ix < p)
    print(ixx)
    return csr.csr_matrix((data[ixx],(row_ix[ixx],col_ix[ixx])),shape=(n,p)),ix


def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)




In [12]:
cols = ['user','item','rating','timestamp']

train = pd.read_csv('/Users/macbookpro/Documents/Lab_Git/tensorflow_practice/recommendation/recommendation-FM-demo/data/ua.base',delimiter='\t',names = cols)
test = pd.read_csv('/Users/macbookpro/Documents/Lab_Git/tensorflow_practice/recommendation/recommendation-FM-demo/data/ua.test',delimiter='\t',names = cols)

x_train,ix = vectorize_dic({'users':train['user'].values,
                            'items':train['item'].values},n=len(train.index),g=2)
x_test,ix = vectorize_dic({'users':test['user'].values,
                           'items':test['item'].values},ix,x_train.shape[1],n=len(test.index),g=2)

print(x_train)
y_train = train['rating'].values
y_test = test['rating'].values

x_train = x_train.todense()
x_test = x_test.todense()

print(x_train)

print(x_train.shape)
print (x_test.shape)

181140 [  1   1   2 ...,  46 158   4]
181140 [    0     0     1 ..., 90568 90569 90569]
2623 {'705items': 128, '932users': 231, '42items': 143, '377users': 23, '448users': 28, '950items': 27, '216items': 266, '400users': 12, '829items': 30, '663users': 148, '1186items': 3, '588users': 215, '305items': 66, '1437items': 9, '58users': 144, '1109items': 26, '537users': 480, '785items': 36, '1145items': 9, '777items': 4, '348items': 23, '185items': 219, '591users': 74, '794users': 29, '500users': 215, '225users': 17, '745items': 14, '505items': 62, '186users': 82, '122items': 100, '1448items': 3, '1305items': 7, '739users': 25, '482users': 16, '137items': 155, '251items': 41, '406users': 332, '978items': 25, '799items': 5, '232items': 93, '1551items': 2, '639users': 138, '1219items': 18, '16users': 130, '126items': 74, '406items': 47, '96users': 46, '539users': 46, '210users': 122, '348users': 48, '753users': 47, '1503items': 10, '662users': 13, '385users': 271, '5users': 165, '1545items': 

18860 [263  62 264 ...,  31 168  44]
18860 [   0    0    1 ..., 9428 9429 9429]
2625 {'705items': 137, '932users': 241, '42items': 148, '377users': 33, '448users': 38, '950items': 30, '216items': 290, '400users': 22, '829items': 34, '663users': 158, '1186items': 3, '588users': 225, '305items': 87, '1437items': 9, '58users': 154, '1109items': 28, '537users': 490, '785items': 39, '1145items': 9, '777items': 4, '348items': 27, '185items': 239, '591users': 84, '794users': 39, '500users': 225, '225users': 27, '745items': 16, '505items': 68, '186users': 92, '122items': 106, '1448items': 3, '1305items': 7, '739users': 35, '482users': 26, '137items': 171, '251items': 46, '406users': 342, '978items': 27, '799items': 5, '232items': 101, '1551items': 2, '639users': 148, '1219items': 20, '16users': 140, '126items': 97, '406items': 49, '96users': 56, '539users': 56, '210users': 132, '348users': 58, '753users': 57, '1503items': 10, '662users': 23, '385users': 281, '5users': 175, '1545items': 4, '702

[[ 0.  2.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(90570, 2623)
(9430, 2623)


In [3]:



n,p = x_train.shape

k = 10

x = tf.placeholder('float',[None,p])

y = tf.placeholder('float',[None,1])

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))

v = tf.Variable(tf.random_normal([k,p],mean=0,stddev=0.01))

#y_hat = tf.Variable(tf.zeros([n,1]))

linear_terms = tf.add(w0,tf.reduce_sum(tf.multiply(w,x),1,keep_dims=True)) # n * 1
pair_interactions = 0.5 * tf.reduce_sum(
    tf.subtract(
        tf.pow(
            tf.matmul(x,tf.transpose(v)),2),
        tf.matmul(tf.pow(x,2),tf.transpose(tf.pow(v,2)))
    ),axis = 1 , keep_dims=True)


y_hat = tf.add(linear_terms,pair_interactions)

lambda_w = tf.constant(0.001,name='lambda_w')
lambda_v = tf.constant(0.001,name='lambda_v')

l2_norm = tf.reduce_sum(
    tf.add(
        tf.multiply(lambda_w,tf.pow(w,2)),
        tf.multiply(lambda_v,tf.pow(v,2))
    )
)

error = tf.reduce_mean(tf.square(y-y_hat))
loss = tf.add(error,l2_norm)


train_op = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)


epochs = 10
batch_size = 1000

# Launch the graph
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in tqdm(range(epochs), unit='epoch'):
        perm = np.random.permutation(x_train.shape[0])
        # iterate over batches
        for bX, bY in batcher(x_train[perm], y_train[perm], batch_size):
            _,t = sess.run([train_op,loss], feed_dict={x: bX.reshape(-1, p), y: bY.reshape(-1, 1)})
            print(t)


    errors = []
    for bX, bY in batcher(x_test, y_test):
        errors.append(sess.run(error, feed_dict={x: bX.reshape(-1, p), y: bY.reshape(-1, 1)}))
        print(errors)
    RMSE = np.sqrt(np.array(errors).mean())
    print (RMSE)

  (0, 1)	2.0
  (1, 1)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (2, 3)	1.0
  (3, 1)	1.0
  (3, 4)	1.0
  (4, 1)	1.0
  (4, 5)	1.0
  (5, 1)	1.0
  (5, 6)	1.0
  (6, 1)	1.0
  (6, 7)	1.0
  (7, 1)	1.0
  (7, 8)	1.0
  (8, 1)	1.0
  (8, 9)	1.0
  (9, 1)	1.0
  (9, 10)	1.0
  (10, 1)	1.0
  (10, 11)	1.0
  (11, 1)	1.0
  (11, 12)	1.0
  (12, 1)	1.0
  (12, 13)	1.0
  :	:
  (90557, 146)	1.0
  (90558, 51)	1.0
  (90558, 147)	1.0
  (90559, 96)	1.0
  (90559, 148)	1.0
  (90560, 44)	1.0
  (90560, 149)	1.0
  (90561, 38)	1.0
  (90561, 150)	1.0
  (90562, 90)	1.0
  (90562, 151)	1.0
  (90563, 133)	1.0
  (90563, 152)	1.0
  (90564, 38)	1.0
  (90564, 153)	1.0
  (90565, 115)	1.0
  (90565, 154)	1.0
  (90566, 71)	1.0
  (90566, 155)	1.0
  (90567, 41)	1.0
  (90567, 156)	1.0
  (90568, 46)	1.0
  (90568, 157)	1.0
  (90569, 4)	1.0
  (90569, 158)	1.0
[[ 0.  2.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  

13.6236
13.1047
12.5847
12.736
11.4308
11.5647
10.8447
10.632
10.3845
9.20325
9.64259
9.16006
8.76089
8.61368
7.97011
8.15394
7.47999
7.48789
7.16596
6.97665
6.54898
6.16551
6.38799
5.68104
5.99215
5.6804
5.46853
5.46394
5.24609
5.07216
5.00078
4.697
4.68722
4.57986
4.21117
4.15247
4.21973
4.01742
3.92202
3.78237
3.56096
3.53148
3.32603
3.15472
3.35881
3.21887
3.17315
2.98264
2.97366
2.7619
2.91412
2.75449
2.72759
2.61522
2.50575
2.6606
2.42602
2.54056
2.40632
2.33074
2.34409
2.2501
2.2062
2.30523
2.22355
2.10251
1.98715
2.02659
2.01722
1.95219
1.99849
1.9881
2.05364
1.99524
1.84384
1.72929
1.7336
1.78619
1.84003
1.74338
1.77738
1.75516
1.6484
1.71948
1.8096
1.58555
1.71198
1.60611
1.58027
1.64584
1.64798
1.52954
1.59635
1.63917
1.50611
1.47844
1.52141
1.57433
1.46684
1.38352
1.39818
1.40208
1.48979
1.448
1.44868
1.54828
1.49759
1.45274
1.4511
1.45217
1.42958
1.44268
1.38753
1.39084
1.28326
1.30662
1.34678
1.35602
1.37215
1.37296
1.38031
1.30119
1.33715
1.38195
1.37656
1.36545
1.39785
